In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e9/sample_submission.csv
/kaggle/input/playground-series-s4e9/train.csv
/kaggle/input/playground-series-s4e9/test.csv
/kaggle/input/lgbm-cat-bagging/__results__.html
/kaggle/input/lgbm-cat-bagging/submission.csv
/kaggle/input/lgbm-cat-bagging/__notebook__.ipynb
/kaggle/input/lgbm-cat-bagging/__output__.json
/kaggle/input/lgbm-cat-bagging/custom.css
/kaggle/input/lgbm-cat-bagging/AutogluonModels/ag-20240929_172922/version.txt
/kaggle/input/lgbm-cat-bagging/AutogluonModels/ag-20240929_172922/metadata.json
/kaggle/input/lgbm-cat-bagging/AutogluonModels/ag-20240929_172922/learner.pkl
/kaggle/input/lgbm-cat-bagging/AutogluonModels/ag-20240929_172922/predictor.pkl
/kaggle/input/lgbm-cat-bagging/AutogluonModels/ag-20240929_172922/utils/data/X.pkl
/kaggle/input/lgbm-cat-bagging/AutogluonModels/ag-20240929_172922/utils/data/y.pkl
/kaggle/input/lgbm-cat-bagging/AutogluonModels/ag-20240929_172922/models/trainer.pkl
/kaggle/input/lgbm-cat-bagging/AutogluonModels/a

In [2]:
pip install -U autogluon --ignore-installed tbb --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134

In [3]:
from autogluon.tabular import TabularPredictor

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import re

import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping
from catboost import CatBoostRegressor, Pool

from xgboost import XGBRegressor


import random

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from autogluon.tabular import TabularPredictor

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
Original = pd.read_csv('/kaggle/input/used-car-price-prediction-dataset/used_cars.csv')
Original[['milage', 'price']] = Original[['milage', 'price']].map(
    lambda x: int(''.join(re.findall(r'\d+', x))))


train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)


train = pd.concat([train, Original], ignore_index=True)

In [5]:
list(set(train['model'].unique()) - set(test['model'].unique()))
train = train[~train['model'].isin(list(set(train['model'].unique()) - set(test['model'].unique())))]

In [6]:
def extract_age_features(df):
    current_year = 2024

    df['Vehicle_Age'] = current_year - df['model_year']
    
    df['Mileage_per_Year'] = df['milage'] / df['Vehicle_Age']
    df['milage_with_age'] =  df.groupby('Vehicle_Age')['milage'].transform('mean')
    
    df['Mileage_per_Year_with_age'] =  df.groupby('Vehicle_Age')['Mileage_per_Year'].transform('mean')
    
    return df

def segment_mileage(df):
    def popka(milage):
        if milage < 70000:
            return 'Low'
        elif milage < 150000:
            return 'Medium'
        elif milage < 190000:
            return 'High'
        else: return 'pizda'
    df['popa'] = df['milage'].apply(popka)
    return df
    
def extract_other_features(df):
    
    luxury_brands =  ['Bugatti', 'Lamborghini', 'Rolls-Royce', 'Bentley', 'McLaren', 'Ferrari','Aston']
    df['Is_Luxury_Brand'] = df['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

    return df

train = extract_age_features(train)
test = extract_age_features(test)

train = segment_mileage(train)
test = segment_mileage(test)

train = extract_other_features(train)
test = extract_other_features(test)

In [7]:
df = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv", index_col= 'id')

In [8]:
columns_to_match=['brand', 'model', 'model_year', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
        'Vehicle_Age', 'popa', 'Is_Luxury_Brand']

intersection = pd.merge(train, test, how='inner',on=columns_to_match, indicator=True)

# Оставляем только те строки, которые есть в обоих датафреймах
intersection = intersection[intersection['_merge'] == 'both'].drop(columns='_merge')
columns_to_match=['brand', 'model', 'model_year', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
        'Vehicle_Age', 'popa', 'Is_Luxury_Brand']
#ищем дорогие тачки
intersection = pd.merge(train, test, how='inner',on=columns_to_match, indicator=True)
def update(df):
    
    t = 100
    
    cat_c = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title','popa',
             
            ]
    re_ = ['model','engine','transmission','ext_col','int_col']
    
    for col in re_:
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < t, col] = "noise"
        
    for col in cat_c:
        df[col] = df[col].fillna('missing')
        df[col] = df[col].astype('category')
        
    return df

train  = update(train)
test   = update(test)

X = train.drop('price', axis=1)
y = train['price']

In [9]:
import numpy as np
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold



callbacks = [log_evaluation(period=300), early_stopping(stopping_rounds=200)]

cat_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"cat_cols--------{cat_cols}")


def get_MAE_oof(df, target, lgb_params, cat_params=None, model_type='LGBM'):

    
    oof_predictions = np.zeros(len(df))
    kf = KFold(n_splits=5, shuffle=True, random_state=1)
    models = []
    rmse_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        print(f"Training fold {fold + 1}/{5} with {model_type}")

        X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]

        if model_type == 'LGBM':
            train_data = lgb.Dataset(X_train, label=y_train)
            val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
            
            model = lgb.train(
                lgb_params,
                train_data,
                valid_sets=[train_data, val_data],
                valid_names=['train', 'valid'],
                callbacks=callbacks    
            )
        
        elif model_type == 'CAT':
            train_data = Pool(data=X_train, label=y_train , cat_features=cat_cols)
            val_data = Pool(data=X_val, label=y_val , cat_features=cat_cols )
            
            model = CatBoostRegressor(**cat_params)
            model.fit(train_data, eval_set=val_data, verbose=150, early_stopping_rounds=200)
        
        models.append(model)
        
        if model_type == 'LGBM':
            pred = model.predict(X_val, num_iteration=model.best_iteration)
        elif model_type == 'CAT':
            pred = model.predict(X_val)
        
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        rmse_scores.append(rmse)

        print(f'{model_type} Fold RMSE: {rmse}')
        
        oof_predictions[val_idx] = pred
        
    print(f'Mean RMSE: {np.mean(rmse_scores)}')
    return oof_predictions, models




lgb_params = {
    'objective': 'MAE',
    'n_estimators': 1000,
    'random_state': 1,
}

oof_predictions_lgbm, models_lgbm = get_MAE_oof(X, y, lgb_params, model_type='LGBM')
X['LGBM_MAE'] = oof_predictions_lgbm


LGBM_preds = np.zeros(len(test))
for model in models_lgbm:
    LGBM_preds += model.predict(test) / len(models_lgbm)
test['LGBM_MAE'] = LGBM_preds



lgb_params = {
    'objective': 'MSE',
    'n_estimators': 1000,
    'random_state': 1,
}

oof_predictions_lgbm, models_lgbm = get_MAE_oof(X, y, lgb_params, model_type='LGBM')

X['LGBM_MSE_diff'] = oof_predictions_lgbm - X['LGBM_MAE']


LGBM_preds = np.zeros(len(test))
for model in models_lgbm:
    LGBM_preds += model.predict(test) / len(models_lgbm)
test['LGBM_MSE_diff'] = LGBM_preds - test['LGBM_MAE']

test.head()

cat_cols--------['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'popa']
Training fold 1/5 with LGBM
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022418 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1729
[LightGBM] [Info] Number of data points in the train set: 154013, number of used features: 17
[LightGBM] [Info] Start training from score 30775.000000
Training until validation scores don't improve for 200 rounds
[300]	train's l1: 16404.3	valid's l1: 16781.3
[600]	train's l1: 16192.2	valid's l1: 16779
Early stopping, best iteration is:
[601]	train's l1: 16190.1	valid's l1: 16778.2
LGBM Fold RMSE: 67864.71192678

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,Vehicle_Age,Mileage_per_Year,milage_with_age,Mileage_per_Year_with_age,popa,Is_Luxury_Brand,LGBM_MAE,LGBM_MSE_diff
0,Land,noise,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes,9,10888.888889,81078.503981,9008.722665,Medium,0,16561.651566,3839.718614
1,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes,4,2285.500000,34258.886442,8564.721611,Low,0,57042.320947,15610.873177
2,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,missing,2,14060.500000,17877.043403,8938.521702,Low,0,48870.687359,9517.211084
3,Audi,noise,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,noise,Black,None reported,missing,8,7657.250000,75999.679762,9499.959970,Low,0,25378.794844,3452.569548
4,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,6,9833.333333,52105.532436,8684.255406,Low,0,27080.066657,4590.180662


In [10]:
X['price'] = y

predictor = TabularPredictor(label='price',
                            eval_metric='rmse',
                            problem_type='regression').fit(X,
                                                       presets='best_quality',
                                                       time_limit=3600*1,
                                                       verbosity=2,
                                                       num_gpus=0,
                                                       included_model_types=['GBM', 'CAT'])

No path specified. Models will be saved in: "AutogluonModels/ag-20240929_214209"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       29.98 GB / 31.36 GB (95.6%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets o

In [11]:
y_pred = predictor.predict(test)

sub_blend = pd.read_csv('/kaggle/input/goodluck2/submission_9.csv')
sample_sub = pd.read_csv('/kaggle/input/goodluck2/sample_submission.csv')
sample_sub['price'] =  y_pred * 0.50 + sub_blend['price'] * 0.50 
sample_sub.to_csv("submission.csv", index=False)
sample_sub.head()

,id,price
0,188533,19142.844547
1,188534,77929.276288
2,188535,57310.454533
3,188536,30538.442072
4,188537,30902.855387
